In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
!ls

/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
id12.0.txt  id25.0.txt	id28.0.txt  id32.0.txt	id36.0.txt  id6.0.txt
id17.0.txt  id26.0.txt	id3.0.txt   id33.0.txt	id45.0.txt  id9.0.txt


In [ ]:
def linear_equation(x1,y1,x2,y2):
    a = (y1-y2)/(x1-x2)
    c = y1-a*x1
    b = -1
    return a, b, c

In [ ]:
def safe(value1, value2):
    if value1*value2>=0:
        return True
    return False

In [ ]:
#Hà Nam
#Đường thẳng phân biệt vượt 
x1 = 935.2941176470588
y1 = 725.235294117647
x2 = 1752.941176470588
y2 = 725.235294117647

# Điểm để phân loại
X = 1350
Y = 500

# Đường thẳng phân chia
a1, b1, c1 = linear_equation(x1,y1,x2,y2)

In [ ]:
#Thái Nguyên
#Đường thẳng phân biệt vượt 
x3 = 803.0
y3 = 582.0
x4 = 2541.0
y4 = 582.0

# Điểm để phân loại
X1 = 1360
Y1 = 400

# Đường thẳng phân chia
a2, b2, c2 = linear_equation(x3,y3,x4,y4)

In [ ]:
#Vũng Tàu
#Đường thẳng phân biệt vượt 
x5 = 807.0
y5 = 658.0
x6 = 2361.0
y6 = 658.0

# Điểm để phân loại
X2 = 1360
Y2 = 400

# Đường thẳng phân chia
a3, b3, c3 = linear_equation(x5,y5,x6,y6)

In [2]:
import os
import math
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

class MyCustomDataset(Sequence):
    def __init__(self, info_files, batch_size=1, max_len=147):
        """Instaniate a custom dataset.

        Attributes
          info_files (list or str): List of info files or only one file. Each file
            should have pattern like 'DendoX/*/info.txt' or 'DenxanhX/*/info.txt'.
          batch_size (int): Batch size.
          max_len (int): Maximum of length of sequence.
        """
        self.batch_size = batch_size
        self.max_len = max_len
        if isinstance(info_files, str):
            info_files = [info_files]
        
        for path in info_files:
            assert os.path.isfile(path), f'Not found info file: {path}'
        self.data = self._load_data(info_files)
    
    def on_epoch_end(self):
        np.random.shuffle(self.data)

    def _load_data(self, file_paths):
        data = []
        for path in file_paths:
            # Load label from file path.
           
            # Store object moving states in a list. So this should be a list of
            # 4-value list.
            sequences = []
            central_list = []
            bbox_list = []
            with open(path) as f:
                for line in f:
                    line = line.strip()
                    row = line.split(',')
                    #row = list(map(float, row))  # convert to float
                    bboxes = row[-4:]
                    #store coordinate central point
                    central = []
                    central.append((float(bboxes[0])+float(bboxes[1]))/2)
                    central.append((float(bboxes[2])+float(bboxes[3]))/2)  
                    #normalize bbox
                    """
                    bboxes_normalize = []
                    bboxes_normalize.append(float(bboxes[0])/2560)
                    bboxes_normalize.append(float(bboxes[1])/1920)
                    bboxes_normalize.append(float(bboxes[2])/2560)
                    bboxes_normalize.append(float(bboxes[3])/1920)
                    """
                    sequences.append(bboxes)
                    central_list.append(central)
                    bbox_list.append(bboxes)
            #process label
            name_province = path.split('/')[-2]
            check = 0
            if 'hanam' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 797:
                    check += 1
            if 'thainguyen' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 839:
                    check += 1
            if 'vungtau' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 727:
                    check += 1
            if (check >= 1): 
              label = 1 #vuot den do
            if (check == 0):
              label = 0 #khong vuot
            #padding 
            #sequences_pad = keras.preprocessing.sequence.pad_sequences([sequences], maxlen=147, padding="pre")
            while len(sequences) < 147:
              sequences.insert(0, [0,0,0,0])

            sequences_pad = sequences[:self.max_len]
            data.append((sequences_pad, label))
        return data
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def __getitem__(self, index):
        """Returns a batch of shape (N, S, 4)"""
        batch_data = self.data[index:self.batch_size:(index+1)*self.batch_size]
        X_batch = np.empty((self.batch_size, self.max_len, 4))
        y_batch = []
        for sequences, label in batch_data:
            X_batch[:len(sequences), :, :] =  np.array(sequences, dtype='float32')
            y_batch.append(label)
        X_batch = np.array(X_batch, dtype='float32')
        y_batch = np.array(y_batch, dtype='int32')
        return X_batch, y_batch

In [3]:
# sắp xếp tên thư mục giống trên window
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [4]:
path = '/content/drive/MyDrive/Project/data_test/dendo/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['vungtau_59.0.avi_save',
 'vungtau_64.0.avi_save',
 'vungtau_65.0.avi_save',
 'vungtau_69.0.avi_save',
 'vungtau_70.0.avi_save',
 'vungtau_75.0.avi_save',
 'vungtau_76.0.avi_save',
 'vungtau_81.0.avi_save',
 'vungtau_82.0.avi_save',
 'vungtau_87.0.avi_save',
 'vungtau_92.0.avi_save',
 'vungtau_93.0.avi_save',
 'vungtau_98.0.avi_save',
 'vungtau_99.0.avi_save',
 'vungtau_103.0.avi_save',
 'vungtau_104.0.avi_save',
 'vungtau_109.0.avi_save',
 'vungtau_110.0.avi_save',
 'vungtau_120.0.avi_save',
 'vungtau_121.0.avi_save',
 'vungtau_126.0.avi_save',
 'vungtau_127.0.avi_save',
 'vungtau_132.0.avi_save',
 'vungtau_133.0.avi_save',
 'vungtau_137.0.avi_save',
 'vungtau_138.0.avi_save',
 'vungtau_143.0.avi_save',
 'vungtau_144.0.avi_save',
 'vungtau_149.0.avi_save',
 'vungtau_150.0.avi_save',
 'vungtau_161.0.avi_save',
 'vungtau_166.0.avi_save',
 'vungtau_167.0.avi_save',
 'vungtau_171.0.avi_save',
 'vungtau_172.0.avi_save',
 'vungtau_177.0.avi_save',
 'vungtau_178.0.avi_save',
 'vungtau_183.0

In [5]:
dataset = []
label = []
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)
"""
info_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'
dataset = MyCustomDataset(info_files)
for batch in dataset:
    print(batch[0].shape, batch[1].shape)
    break

print(batch[0])
"""

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

"\ninfo_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'\ndataset = MyCustomDataset(info_files)\nfor batch in dataset:\n    print(batch[0].shape, batch[1].shape)\n    break\n\nprint(batch[0])\n"

In [ ]:
label

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], 

In [ ]:
a = [[1,2,3,4],[1,2,3,4]]
a.append([0,0,0,0])
a = keras.preprocessing.sequence.pad_sequences([a], maxlen=147, padding="post")
a
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(a)
print(masked_output)
print(masked_output._keras_mask)

In [6]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)

In [7]:
print(data.shape)
print(label_data.shape)

(477, 147, 4)
(477,)


In [ ]:
data[0]

array([[1439.51  , 1271.19  ],
       [1448.315 , 1251.405 ],
       [1454.695 , 1225.3235],
       [1463.71  , 1211.5105],
       [1470.745 , 1192.2985],
       [1486.165 , 1172.103 ],
       [1489.75  , 1154.248 ],
       [1498.915 , 1130.574 ],
       [1507.535 , 1116.478 ],
       [1512.14  , 1102.242 ],
       [1521.06  , 1082.1895],
       [1525.795 , 1077.2505],
       [1537.54  , 1060.1365],
       [1541.37  , 1048.1985],
       [1547.805 , 1040.2885],
       [1557.25  , 1025.702 ],
       [1564.49  , 1015.5845],
       [1568.88  , 1006.2715],
       [1571.125 ,  998.0575],
       [1582.985 ,  989.447 ],
       [1587.52  ,  978.006 ],
       [1592.    ,  971.6495],
       [1597.82  ,  962.6615],
       [1601.19  ,  954.965 ],
       [1604.58  ,  948.292 ],
       [1609.745 ,  940.7165],
       [1611.895 ,  936.6895],
       [1618.25  ,  927.74  ],
       [1622.16  ,  924.658 ],
       [1626.08  ,  920.632 ],
       [1629.08  ,  912.6825],
       [1632.71  ,  908.6195],
       [

In [ ]:
label

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], 

In [8]:
path = '/content/drive/MyDrive/Project/data_test/denxanh/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['hanam_1.0.avi_save',
 'hanam_4.0.avi_save',
 'hanam_30.0.avi_save',
 'hanam_33.0.avi_save',
 'hanam_59.0.avi_save',
 'hanam_62.0.avi_save',
 'hanam_88.0.avi_save',
 'hanam_91.0.avi_save']

In [9]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [10]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)
print(data.shape)
print(label_data.shape)


(546, 147, 4)
(546,)


In [ ]:
label

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], 

In [ ]:
cnt_0 = 0
cnt_1 = 0
for i in label:
  if i == 0:
    cnt_0 += 1
  else:
    cnt_1 += 1
print(cnt_0)
print(cnt_1)

423
123


In [ ]:
data_final = np.concatenate([data, label_data], axis=1) 

ValueError: ignored

####Ghi ra file csv

In [11]:
#ghi ra file data
import csv
b = open('/content/drive/MyDrive/Project/data_test.csv', 'w')
a = csv.writer(b)
a.writerows(data)
b.close()

In [12]:
#ghi ra file label
np.savetxt('/content/drive/MyDrive/Project/label_test.csv', label_data, delimiter=",", fmt='%d')